In [1]:
import sys, os
sys.path.append(os.pardir)

import gfootball.env as football_env
from api.internal.tamakeri import TamakeriAgent
from api.internal.grf import GrfAgent


Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.





In [14]:
grf_agent = GrfAgent()
tamakeri_agent = TamakeriAgent()

In [22]:
dump_fp = '/app/api/resources/matches/episode_done_20210918-064519149416.dump'
from gfootball.env import script_helpers, observation_preprocessing

dump_raw = script_helpers.ScriptHelpers().load_dump(dump_fp)
for frame in dump_raw:
    break
    
def _convert_observation(observation_raw):
    observation_raw['active'] = int(observation_raw['left_team_designated_player'])  # for grf
    observation_raw['sticky_actions'] = observation_raw['left_agent_sticky_actions']  # for tamakeri
    return [observation_raw]

In [30]:
def _convert_observation(observation_raw):
    observation_raw['active'] = int(observation_raw['left_team_designated_player'])  # for grf
    observation_raw['sticky_actions'] = observation_raw['left_agent_sticky_actions'][0]  # for tamakeri
    return [observation_raw]

In [31]:
obs = _convert_observation(frame['observation'])
grf_agent.step(obs)

5

In [27]:
from api.internal.tamakeri.env import to_tamakeri_obs

In [32]:
to_tamakeri_obs(obs)

{'controlled_players': 1,
 'players_raw': [{'ball': [0.0, -0.0, 0.11059734225273132],
   'ball_direction': [-0.0, 0.0, -0.0019182914402335882],
   'ball_rotation': [0.0, -0.0, 0.0],
   'left_team': [[-1.0110293626785278, -0.0],
    [-0.048872511833906174, 1.3186933316144955e-11],
    [-0.010110294446349144, 0.21961753070354462],
    [-0.4253305196762085, -0.19827446341514587],
    [-0.5051397085189819, -0.0642440989613533],
    [-0.5051285624504089, 0.06453930586576462],
    [-0.4266543984413147, 0.19894461333751678],
    [-0.18624374270439148, -0.10739918798208237],
    [-0.270525187253952, -0.0],
    [-0.18624374270439148, 0.10739918798208237],
    [-0.010107884183526039, -0.21957246959209442]],
   'left_team_direction': [[0.0, -0.0],
    [0.0035862713120877743, 2.5969120950875002e-11],
    [0.0, -0.0],
    [0.002783536911010742, 0.0014629090437665582],
    [0.0015041407896205783, 0.0013856797013431787],
    [0.0019227757584303617, -0.0002725842932704836],
    [0.0, -0.0],
    [0.0, 

In [33]:
tamakeri_agent.step(obs)

6

In [ ]:
agent = TamakeriAgent()
env = football_env.create_environment(
        env_name='11_vs_11_easy_stochastic',
        representation='raw',
        number_of_left_players_agent_controls=1,
        number_of_right_players_agent_controls=0
)

In [ ]:
obs, _, _, _ = env.step(agent.get_action())
agent.step(obs)

In [ ]:
sum(agent.get_action_probs(to_name=True).values())

In [ ]:
agent.value[0]

In [ ]:
from api.internal.tamakeri.env import Action

In [ ]:
agent.logit

In [ ]:
env = football_env.create_environment(
        env_name='11_vs_11_easy_stochastic',
        representation='raw',
        number_of_left_players_agent_controls=1,
        number_of_right_players_agent_controls=0,
        logdir='/app/logs',
        render=True
)

tamakeri_env = TamakeriEnv()
model_path = '/app/api/resources/models/1679.pth'
model = load_model(tamakeri_env.net()(tamakeri_env), model_path)
model.eval()

prev_action = 0
reserved_action = None
while True:
    obs, reward, done, info = env.step([prev_action])
    if done:
        break

    info = [{'observation': to_tamakeri_obs(obs), 'action': [prev_action]}, None]
    tamakeri_env.play_info(info)
    x = tamakeri_env.observation(0)

    p, v, r, _ = model.inference(x, None)
    actions = tamakeri_env.legal_actions(0)

    ap_list = sorted([(a, p[a]) for a in actions], key=lambda x: -x[1])
    action = ap_list[0][0]

    if reserved_action is not None:
        prev_action = reserved_action
        reserved_action = None
    else:
        # split action
        prev_action, reserved_action = tamakeri_env.special_to_actions(action)
    break

In [ ]:
obs

In [ ]:
len(p)

In [ ]:
for p_ in p:
    print(p_)

In [ ]:
for o in obs:
    if 'frame' in o:
        del o['frame']

In [ ]:
to_tamakeri_obs(obs)

In [ ]:
import torch

def load_model(model, model_path):
    loaded_dict_ = torch.load(model_path)
    model_dict = model.state_dict()
    loaded_dict = {k: v for k, v in loaded_dict_.items() if k in model_dict}
    model_dict.update(loaded_dict)
    model.load_state_dict(model_dict)
    return model

In [ ]:
from api.internal.tamakeri import Environment as TamakeriEnv
from api.internal.tamakeri.env import OBS_TEMPLATE

In [ ]:
tamakeri_env = TamakeriEnv()
model_path = '/app/api/resources/models/1679.pth'
model = load_model(tamakeri_env.net()(tamakeri_env), model_path)
model.eval()

In [ ]:
import gfootball.env as football_env

In [ ]:
grf_env = football_env.create_environment(
        env_name='11_vs_11_easy_stochastic',
        representation='raw',
        number_of_left_players_agent_controls=1,
        number_of_right_players_agent_controls=0
    )

In [ ]:
grf_obs, reward, done, info = grf_env.step([0])

In [ ]:
def to_tamakeri_obs(grf_obs):
    tamakeri_obs = {
        'controlled_players': grf_obs[0]['active'],
        'players_raw': to_list(grf_obs)
    }
    return tamakeri_obs

In [ ]:
from api.internal.tamakeri.model import map_r

def to_list(x):
    return map_r(x, lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

In [ ]:
to_list(grf_obs)

In [ ]:
type(grf_obs[0]['ball'])

isinstance(grf_obs[0]['ball'], np.ndarray)

In [ ]:
obs[0]['designated']

In [ ]:
t_obs = OBS_TEMPLATE['players_raw'][0]
grf_obs = grf_obs[0]

In [ ]:
t_obs.keys()

In [ ]:
grf_obs.keys()

In [ ]:
grf_obs[0]['left_team'].shape, grf_obs[0]['right_team'].shape

In [ ]:
import numpy as np

np.array(OBS_TEMPLATE['players_raw'][0]['left_team']).shape

In [ ]:
nested_keys = set(['player', 'player_index', 'distance'])

for key in t_x.keys():
    print(key)
    if key in nested_keys:
        for k in t_x[key].keys():
            print(t_x[key][k].shape)
            print(grf_x[key][k].shape)
        print()
    else:
        print(t_x[key].shape)
        print(grf_x[key].shape)
        print()

In [ ]:
tamakeri_env = 

In [ ]:
prev_action = 0

info = [{'observation': to_tamakeri_obs(grf_obs), 'action': [prev_action]}, None]
# info = [{'observation': OBS_TEMPLATE, 'action': [prev_action]}, None]
tamakeri_env.play_info(info)
x = tamakeri_env.observation(0)
p, v, r, _ = model.inference(x, None)
p, v, r, _

In [ ]:
x['distance']['p2bo'].shape

In [ ]:
t_x['distance']['p2bo'].shape

In [ ]:
t_x = x
t_x

In [ ]:
for key in t_x.keys():
    print(key)
    print(t_x[key])
    print(grf_x[key])

In [ ]:
grf_x = x
grf_x

In [ ]:
OBS_TEMPLATE['players_raw'][0]['left_team']

In [ ]:
grf_obs[0]['left_team'Developer

In [ ]:
x['distance']

In [ ]:
obs[0]

In [ ]:
OBS_TEMPLATE

In [ ]:
player_state['observation']

In [ ]:
obs

In [ ]:
info